In [7]:
import os
import csv
import numpy as np
from ultralytics import YOLO  

def test_validation(weights_folder, output_csv, data_path):
    # Initialize an empty list to store results
    metrics_data = []

    # Iterate through all the model files in the folder
    for model_file in os.listdir(weights_folder):
        if model_file.endswith(".pt"):  # Ensure it's a YOLO weight file
            model_path = os.path.join(weights_folder, model_file)
            print(f"Validating model: {model_file}")
            
            # Load the model and validate
            try:
                model = YOLO(model_path)  # Load YOLO model
                metrics = model.val(data=data_path, name = model_file)  # Perform validation
                
                # Append metrics to the list
                metrics_data.append({
                    'model': model_file,
                    'mAP50-95': np.round(metrics.box.map, 2),
                    'mAP50': np.round(metrics.box.map50, 2),
                    'mAP75': np.round(metrics.box.map75, 2),
                    'Precision': np.round(metrics.box.p, 2),
                    'Recall': np.round(metrics.box.r, 2)
                })
            except Exception as e:
                print(f"Error validating model {model_file}: {e}")
                continue

    # Write the results to a CSV file
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["model", "mAP50-95", "mAP50", "mAP75", "Precision", "Recall"])
        writer.writeheader()
        writer.writerows(metrics_data)

    print(f"Validation results saved to {output_csv}")


In [13]:
# Define the folder containing the YOLOv11 weights
weights_folder = "/home/wildmosquit0/git/bestModel/test_different_models"
output_csv = "/home/wildmosquit0/git/bestModel/test_different_models/validation_results.csv"
data_path = "/home/wildmosquit0/git/playfraound/val_models.yaml"  # Path to the dataset file


In [14]:
test_validation(weights_folder,output_csv,data_path)

Validating model: batch_3.pt
Ultralytics 8.3.58 🚀 Python-3.9.21 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /home/wildmosquit0/git/model_exp/setups/seperate/640X640/labels/val... 48 images, 3 backgrounds, 0 corrupt: 100%|██████████| 48/48 [00:00<00:00, 835.04it/s]

val: New cache created: /home/wildmosquit0/git/model_exp/setups/seperate/640X640/labels/val.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.03s/it]


                   all         48        268      0.808      0.619      0.741      0.317
Speed: 1.9ms preprocess, 49.6ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/batch_3.pt3
Validating model: sahi_l_1_more_300e_200.pt
Ultralytics 8.3.58 🚀 Python-3.9.21 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


...

                   all         48        268      0.679      0.701      0.667      0.254
Speed: 2.0ms preprocess, 26.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/sahi_l_1.pt3
Validating model: sahi_l_1_more_300e.pt
Ultralytics 8.3.58 🚀 Python-3.9.21 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


val: Scanning /home/wildmosquit0/git/model_exp/setups/seperate/640X640/labels/val.cache... 48 images, 3 backgrounds, 0 corrupt: 100%|██████████| 48/48 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.41it/s]


                   all         48        268      0.661       0.66       0.67      0.252
Speed: 2.0ms preprocess, 30.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/sahi_l_1_more_300e.pt3
Validating model: 28.01.pt
Ultralytics 8.3.58 🚀 Python-3.9.21 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


val: Scanning /home/wildmosquit0/git/model_exp/setups/seperate/640X640/labels/val.cache... 48 images, 3 backgrounds, 0 corrupt: 100%|██████████| 48/48 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.39it/s]


                   all         48        268      0.548      0.627      0.616      0.268
Speed: 2.3ms preprocess, 31.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/28.01.pt3
Validation results saved to /home/wildmosquit0/git/bestModel/test_different_models/validation_results.csv


In [ ]:
os.getcwd()

NameError: name 'model' is not defined